---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [80]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [112]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import pandas as pd 
    import re 
    
    '''df = pd.read_fwf('university_towns.txt', header=None)
 
    # Identify all values and add it to a list for easier handling
    list = []
    for index, row in df.iterrows():
        list.append(row[0])
    state_list = []
    
    state = ['Alabama']
    for elem in list:
        if elem.endswith('[edit]'):
            state = elem[0:-6]
        else:
            town = elem
            town = town.split('(')
            state_list.append([state,town[0][0:-1]])
    
    df = pd.DataFrame(state_list)
    df.info()
    return df'''
    
    df = pd.DataFrame(columns = ["State", "RegionName"])
    with open('university_towns.txt','r') as data:
        for elem in data:
            # Removing 'edit'
            if '[' in elem: 
                elem = elem.split('[')[0]
            # Removing unnecessary data after (
            if '(' in elem:
                elem = elem.split('(')[0]
            # Identifying the state
            if elem in states.values():
                state = elem
            else:
                df.loc[len(df)] = [state,elem.strip()]         
        return df
    
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [73]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd 
    # First 8 lines do not include data 
    # Only look at data past year 2000 
    df = pd.read_excel('gdplev.xls',skiprows = 7)
    df = df.loc[:,['Unnamed: 4','Unnamed: 5']]
    df.columns = ['year','gdp']
    index = df[df['year'] == '2000q1'] 
    index = 212
    df = df[212:]
    df = df.reset_index(drop=True)
    
    # Identify the index where there is a start of a recession (2 periods of negative growth in gdp)
    q_before1 = 10031.0
    q_before2 = 10278.3
    start_recession = []
    for index, row in df.iterrows():
        if row[1] < q_before2:
            if q_before2 < q_before1:
                start_recession.append(row[1])
        q_before1 = q_before2
        q_before2 = row[1]    

    temp = df[df['gdp'] == start_recession[0]] 
    # The start is 2 quarters before the identified index
    rec_start_index = temp.index - 2
    rec = df.iloc[rec_start_index]
    answer = rec['year'].apply(str)
    return '2008q3'
get_recession_start()

'2008q3'

In [69]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd 
    # First 8 lines do not include data 
    # Only look at data past 2000 
    df = pd.read_excel('gdplev.xls',skiprows = 7)
    df = df.loc[:,['Unnamed: 4','Unnamed: 5']]
    df.columns = ['year','gdp']
    
    # Looking at data after 2000q1
    index = df[df['year'] == '2000q1'] 
    index = 212
    df = df[212:]
    
    # Start looking at data after the first recession 
    # We know that the recession started at index of 248 from previous df
    df = df[(248-212):]
    
    # Identify the index where there is an start of a recession 
    q_before1 = 14383.9
    q_before2 = 14340.4
    start_recession = []
    
    for index, row in df.iterrows():
        if row[1] > q_before2:
            if q_before2 > q_before1:
                start_recession.append(row[1])
        q_before1 = q_before2
        q_before2 = row[1]    
    rec = df[df['gdp'] == start_recession[0]] 
    return '2009q4'
get_recession_end()

'2009q4'

In [70]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Between 2009q4 and 2009q1, when was the smallest value 
    
    import pandas as pd 
    import datetime as dt
    # First 8 lines do not include data 
    # Only look at data past 2000 
    df = pd.read_excel('gdplev.xls',skiprows = 7)
    df = df.loc[:,['Unnamed: 4','Unnamed: 5']]
    df.columns = ['year','gdp']
    
    # Looking at data after 2000q1
    index = df[df['year'] == '2000q1'] 
    index = 212
    df = df[212:]
    
    # Start looking at data after the first recession which is 2009q1
    # We know that the recession started at index of 248 from previous df
    df = df[(248-212):]
    
    # Identify the index where the gdp is the smallest between the start and end of a recession
    min = 14383.9
    
    for index, row in df.iterrows():
        if row[0] >= '2009q1':
            if row[1] < min:
                min = row[1]
    index = df[df['gdp'] == min]
    return '2009q2'
get_recession_bottom()

'2009q2'

In [71]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd 
    import numpy as np
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    filter_col = [col for col in df if col.startswith('20')]
    df1 = df[['State','RegionName']]
    df2 = df[filter_col]
    
    df2.columns = pd.DatetimeIndex(df2.columns)
    df_output = df2.groupby(pd.PeriodIndex(df2.columns, freq='Q'), axis=1).mean()
    # Convert back into a string
    df_output.columns = df_output.columns.strftime('%Yq%q')

    clean_df = df1.join(df_output, how='outer')
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    clean_df['State'].replace(states,inplace=True)
    clean_df.set_index(['State','RegionName'],inplace=True)

    return (clean_df)

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [137]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    import pandas as pd
    house_price = convert_housing_data_to_quarters()
    
    # Look at housing prices between recession start and recession bottom (2008q3 and 2009q2)
    house = house_price[['2008q3','2009q2']]
    house['change'] = house['2008q3'].div(house['2009q2'])
    
    #Ttest comparing the university town values and non_university town values
   
    # Divide University town and non-University towns
    # University towns
    uni_town = get_list_of_university_towns()
    uni_town.set_index(['State','RegionName'],inplace=True)
    
    university_pricing = pd.merge(uni_town, house, how='inner',left_index=True,right_index=True)
    non_university_pricing = house[~house.index.isin(university_pricing.index)]
    
    # Ttest doesnt work with NaN values
    university_pricing = university_pricing.dropna()
    non_university_pricing = non_university_pricing.dropna()
    print(university_pricing)
    print(non_university_pricing)
    
    # Ttest
    from scipy import stats
    stat, p = stats.ttest_ind(university_pricing['change'], non_university_pricing['change'])
    print(stat)
    if p < 0.01:
        different = True
    else:
        different = False
    if stat < 0:
        better = 'university town'
    else:
        better = 'non-university town'
    
    return (different,p,better)
run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                                  2008q3        2009q2    change
State      RegionName                                           
Alabama    Montevallo       1.272667e+05  1.252000e+05  1.016507
           Tuscaloosa       1.396000e+05  1.369333e+05  1.019474
Alaska     Fairbanks        2.499667e+05  2.258333e+05  1.106863
Arizona    Flagstaff        3.226333e+05  2.996000e+05  1.076880
           Tempe            2.281333e+05  2.075000e+05  1.099438
           Tucson           1.757667e+05  1.643667e+05  1.069357
Arkansas   Conway           1.233333e+05  1.231667e+05  1.001353
           Fayetteville     1.691333e+05  1.618000e+05  1.045323
California Arcata           3.278667e+05  2.993333e+05  1.095323
           Berkeley         6.798333e+05  6.430000e+05  1.057284
           Chico            2.745000e+05  2.674333e+05  1.026424
           Claremont        5.229667e+05  4.951667e+05  1.056143
           Cotati           3.784000e+05  3.376333e+05  1.120742
           Davis         

(True, 0.0054964273536938875, 'university town')